---
<center>
<h1>  alx-low_level_programming </h1>
</center>

---


In [1]:
%%bash
cd /home/elkomy/alx/simple_shell//////

In [3]:
import os
# creating 0x02-shell_redirectionss

folder_path = r'/home/elkomy/alx/simple_shell/'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Create a README.md inside alx-low_level_programming/0x0D-preprocessor
file_path = os.path.join(folder_path, 'README.md')

with open(file_path, 'w') as file:
    file.write("This is a README.md file inside alx-low_level_programming/0x0D-preprocessor")



In [4]:
%%file /home/elkomy/alx/simple_shell//////README.md
This is a README.md

Overwriting /home/elkomy/alx/simple_shell/README.md


###
---
<center>
<h1> 0x16. C - Simple Shell </h1>
</center>

---


###
---
<center>
<h4> 0. Object-like Macro</h4>
</center>

---


#### Create a header file that defines a macro named `SIZE` as an abbreviation for the token `1024`.


```bash
julien@ubuntu:~/0x0c. macro, structures$ cat 0-main.c
#include "0-object_like_macro.h"
#include "0-object_like_macro.h"
#include <stdio.h>

/**
 * main - check the code
 *
 * Return: Always 0.
 */
int main(void)
{
    int s;

    s = 98 + SIZE;
    printf("%d\n", s);
    return (0);
}
julien@ubuntu:~/0x0c. macro, structures$ gcc -Wall -pedantic -Werror -Wextra -std=gnu89 0-main.c -o a
julien@ubuntu:~/0x0c. macro, structures$ ./a 
1122
julien@ubuntu:~/0x0c. macro, structures$ 

```
#### Repo:
* GitHub repository: ``` alx-low_level_programming ```
* Directory: ```0x0D-preprocessor``` 
* File: ```0-object_like_macro.h```

In [18]:
%%file /home/elkomy/alx/simple_shell/0-main.c
#include <stdio.h>

/**
* main - Entry point
*
* Return: Always 0 (Success)
*/
int main(void)
{
printf("Betty would be proud!\n");
return (0);
}


Overwriting /home/elkomy/alx/simple_shell/0-main.c


In [34]:
%%bash
betty 0-main.c



========== 0-main.c ==========
main


In [19]:
!pwd

/home/elkomy/alx/simple_shell


In [36]:
%%bash
gcc -Wall -pedantic -Werror -Wextra -std=gnu89 0-main.c -o a.out
./a.out


Betty would be proud!


In [37]:
%%bash
git add .
git commit -m "alx-low_level_programming/alx/simple_shell0-putchar"
git push

[main add5e92] alx-low_level_programming/alx/simple_shell0-putchar
 4 files changed, 892 insertions(+), 1 deletion(-)
 create mode 100644 0-main.c
 create mode 100644 0x16. C - Simple Shell.ipynb
 create mode 100755 a.out


To https://github.com/elkomysara/simple_shell.git
   5e061ea..add5e92  main -> main


###
---
<center>
<h4>1. Simple shell 0.1</h4>
</center>

---


#### Write a UNIX command line interpreter.

* Usage: `simple_shell`
#### Your Shell should:

* Display a prompt and wait for the user to type a command. A command line always ends with a new line.
* The prompt is displayed again each time a command has been executed.
* The command lines are simple, no semicolons, no pipes, no redirections or any other advanced features.
* The command lines are made only of one word. No arguments will be passed to programs.
* If an executable cannot be found, print an error message and display the prompt again.
* Handle errors.
* You have to handle the “end of file” condition (`Ctrl+D`)
#### You don’t have to:

* use the `PATH`
* implement built-ins
* handle special characters : ``", ', `, \, *, &, #``
* be able to move the cursor
* handle commands with arguments
`execve` will be the core part of your Shell, don’t forget to pass the environ to it…

```bash
julien@ubuntu:~/shell$ ./shell 
#cisfun$ ls
./shell: No such file or directory
#cisfun$ /bin/ls
barbie_j       env-main.c  exec.c  fork.c  pid.c  ppid.c    prompt   prompt.c  shell.c  stat.c         wait
env-environ.c  exec    fork    mypid   ppid   printenv  promptc  shell     stat test_scripting.sh  wait.c
#cisfun$ /bin/ls -l
./shell: No such file or directory
#cisfun$ ^[[D^[[D^[[D
./shell: No such file or directory
#cisfun$ ^[[C^[[C^[[C^[[C
./shell: No such file or directory
#cisfun$ exit
./shell: No such file or directory
#cisfun$ ^C
julien@ubuntu:~/shell$ echo "/bin/ls" | ./shell
barbie_j       env-main.c  exec.c  fork.c  pid.c  ppid.c    prompt   prompt.c  shell.c  stat.c         wait
env-environ.c  exec    fork    mypid   ppid   printenv  promptc  shell     stat test_scripting.sh  wait.c
#cisfun$ julien@ubuntu:~/shell$

```
#### Repo:
* GitHub repository: ``` simple_shell ```

In [2]:
!pwd

/home/elkomy/alx/simple_shell


In [81]:
%%file /home/elkomy/alx/simple_shell/shell.h

#ifndef _SHELL_H_
#define _SHELL_H_

#include <stdio.h>
#include <stdlib.h>
#include <unistd.h>
#include <string.h>
#include <sys/types.h>
#include <sys/wait.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <errno.h>

/* Constants */
#define READ_BUF_SIZE 1024
#define WRITE_BUF_SIZE 1024
#define BUF_FLUSH -1

/* Command chaining */
#define CMD_NORM    0
#define CMD_OR      1
#define CMD_AND     2
#define CMD_CHAIN   3

/* Convert number() */
#define CONVERT_LOWERCASE  1
#define CONVERT_UNSIGNED   2

/* History file */
#define HIST_FILE  ".simple_shell_history"
#define HIST_MAX   4096

extern char **environ;

/**
 * struct liststr - singly linked list
 * @num: the number field
 * @str: a string
 * @next: points to the next node
 */
typedef struct liststr
{
    int num;
    char *str;
    struct liststr *next;
} list_t;

/**
 * struct passinfo - contains pseudo-arguments to pass into a function,
 * allowing uniform prototype for function pointer struct
 * @arg: a string generated from getline containing arguments
 * @argv: an array of strings generated from arg
 * @path: a string path for the current command
 * @argc: the argument count
 * @line_count: the error count
 * @err_num: the error code for exit()s
 * @linecount_flag: if on count this line of input
 * @fname: the program filename
 * @env: linked list local copy of environ
 * @environ: custom modified copy of environ from LL env
 * @history: the history node
 * @alias: the alias node
 * @env_changed: on if environ was changed
 * @status: the return status of the last exec'd command
 * @cmd_buf: address of pointer to cmd_buf, on if chaining
 * @cmd_buf_type: CMD_type ||, &&, ;
 * @readfd: the fd from which to read line input
 * @histcount: the history line number count
 */
typedef struct passinfo
{
    char *arg;
    char **argv;
    char *path;
    int argc;
    unsigned int line_count;
    int err_num;
    int linecount_flag;
    char *fname;
    list_t *env;
    list_t *history;
    list_t *alias;
    char **environ;
    int env_changed;
    int status;

    char **cmd_buf; /* pointer to cmd ; chain buffer, for memory management */
    int cmd_buf_type; /* CMD_type ||, &&, ; */
    int readfd;
    int histcount;
} info_t;

#define INFO_INIT \
{NULL, NULL, NULL, 0, 0, 0, 0, NULL, NULL, NULL, NULL, 0, 0, NULL, 0, 0, 0}

/* Function prototypes from shell_loop.c */
int hsh(info_t *info, char **av);
int find_builtin(info_t *info);
void find_cmd(info_t *info);
void fork_cmd(info_t *info);

/* Function prototypes from getLine.c */
ssize_t get_input(info_t *info);
ssize_t input_buf(info_t *info, char **buf, size_t *len);
ssize_t read_buf(info_t *info, char *buf, size_t *i);
int _getline(info_t *info, char **ptr, size_t *length);
void sigintHandler(int sig_num);

/* Function prototypes from main.c */
int main(int ac, char **av);

/* Function prototypes from the errors.c */
void _eputs(char *str);
int _eputchar(char c);
int _putfd(char c, int fd);
int _putsfd(char *str, int fd);

/* Additional Function prototypes */
int interactive(info_t *info);
int write_history(info_t *info);
int read_history(info_t *info);
int populate_env_list(info_t *info);
void clear_info(info_t *info);
void set_info(info_t *info, char **av);
void free_info(info_t *info, int all);
int _puts(char *str);
int _putchar(char c);
int _strlen(char *s);
int is_chain(info_t *info, char *buf, size_t *j);
void check_chain(info_t *info, char *buf, size_t *j, size_t i, size_t len);

#endif /* _SHELL_H_ */


Overwriting /home/elkomy/alx/simple_shell/shell.h


In [82]:
%%file /home/elkomy/alx/simple_shell/getLine.c
#include "shell.h"

/**
* get_input - gets a line minus the newline
* @info: parameter struct
*
* Return: bytes read
*/
ssize_t get_input(info_t *info)
{
static char *buf; /* the ';' command chain buffer */
static size_t i, j, len;
ssize_t r = 0;
char **buf_p = &(info->arg), *p;

_putchar(BUF_FLUSH);
r = input_buf(info, &buf, &len);
if (r == -1) /* EOF */
return (-1);
if (len)    /* we have commands left in the chain buffer */
{
j = i; /* init new iterator to current buf position */
p = buf + i; /* get pointer for return */

check_chain(info, buf, &j, i, len);
while (j < len) /* iterate to semicolon or end */
{
if (is_chain(info, buf, &j))
break;
j++;
}

i = j + 1; /* increment past nulled ';'' */
if (i >= len) /* reached end of buffer? */
{
i = len = 0; /* reset position and length */
info->cmd_buf_type = CMD_NORM;
}

*buf_p = p; /* pass back pointer to current command position */
return (_strlen(p)); /* return length of current command */
}

*buf_p = buf; /* else not a chain, pass back buffer from _getline() */
return (r); /* return length of buffer from _getline() */
}


Overwriting /home/elkomy/alx/simple_shell/getLine.c


In [108]:
%%file /home/elkomy/alx/simple_shell/main.c
#include "shell.h"

/**
* main - entry point
* @ac: arg count
* @av: arg vector
*
* Return: 0 on success, 1 on error
*/
int main(int ac, char **av)
{
info_t info = {
NULL,  /* arg */
NULL,  /* argv */
NULL,  /* path */
0,     /* argc */
0,     /* line_count */
0,     /* err_num */
0,     /* linecount_flag */
NULL,  /* fname */
NULL,  /* env */
NULL,  /* history */
NULL,  /* alias */
NULL,  /* environ */
0,     /* env_changed */
0,     /* status */
NULL,  /* cmd_buf */
0,     /* cmd_buf_type */
0,     /* readfd */
0      /* histcount */
};

int fd = 2;

asm ("mov %1, %0\n\t"
"add $3, %0"
: "=r" (fd)
: "r" (fd));

if (ac == 2)
{
fd = open(av[1], O_RDONLY);
if (fd == -1)
{
if (errno == EACCES)
exit(126);
if (errno == ENOENT)
{
_eputs(av[0]);
_eputs(": 0: Can't open ");
_eputs(av[1]);
_eputchar('\n');
_eputchar(BUF_FLUSH);
exit(127);
}
return (EXIT_FAILURE);
}
info.readfd = fd;
}
populate_env_list(&info);
read_history(&info);
hsh(&info, av);
return (EXIT_SUCCESS);
}


Overwriting /home/elkomy/alx/simple_shell/main.c


In [109]:
%%file /home/elkomy/alx/simple_shell/shell_loop.c
#include "shell.h"

/**
* hsh - main shell loop
* @info: the parameter & return info struct
* @av: the argument vector from main()
*
* Return: 0 on success, 1 on error, or error code
*/
int hsh(info_t *info, char **av)
{
ssize_t r = 0;
int builtin_ret = 0;

while (r != -1 && builtin_ret != -2)
{
clear_info(info);
if (interactive(info))
_puts("$ ");
_eputchar(BUF_FLUSH);
r = get_input(info);
if (r != -1)
{
set_info(info, av);
builtin_ret = find_builtin(info);
if (builtin_ret == -1)
find_cmd(info);
}
else if (interactive(info))
_putchar('\n');
free_info(info, 0);
}
write_history(info);
free_info(info, 1);
if (!interactive(info) && info->status)
exit(info->status);
if (builtin_ret == -2)
{
if (info->err_num == -1)
exit(info->status);
exit(info->err_num);
}
return (builtin_ret);
}


Overwriting /home/elkomy/alx/simple_shell/shell_loop.c


In [110]:
%%bash
gcc -Wall -Werror -Wextra -pedantic -std=gnu89 /home/elkomy/alx/simple_shell/*.c -o simple_shell
./simple_shell
/bin/ls



/usr/bin/ld: /tmp/ccSfdOv0.o: in function `get_input':
getLine.c:(.text+0x26): undefined reference to `_putchar'
/usr/bin/ld: getLine.c:(.text+0x40): undefined reference to `input_buf'
/usr/bin/ld: getLine.c:(.text+0xb8): undefined reference to `check_chain'
/usr/bin/ld: getLine.c:(.text+0xd7): undefined reference to `is_chain'
/usr/bin/ld: getLine.c:(.text+0x163): undefined reference to `_strlen'
/usr/bin/ld: /tmp/ccyQYkw0.o: in function `main':
main.c:(.text+0xc6): undefined reference to `_eputs'
/usr/bin/ld: main.c:(.text+0xd2): undefined reference to `_eputs'
/usr/bin/ld: main.c:(.text+0xe8): undefined reference to `_eputs'
/usr/bin/ld: main.c:(.text+0xf2): undefined reference to `_eputchar'
/usr/bin/ld: main.c:(.text+0xfc): undefined reference to `_eputchar'
/usr/bin/ld: main.c:(.text+0x122): undefined reference to `populate_env_list'
/usr/bin/ld: main.c:(.text+0x12e): undefined reference to `read_history'
/usr/bin/ld: /tmp/cco4kmTX.o: in function `hsh':
shell_loop.c:(.text+0x30):

0x16. C - Simple Shell.ipynb
README.md
getLine.c
main.c
shell.h
shell_loop.c


In [50]:
%%bash
git add .
git commit -m "alx-low_level_programming/alx/simple_shell0-putchar"
git push

[main 56fc096] alx-low_level_programming/alx/simple_shell0-putchar
 3 files changed, 17 insertions(+), 6 deletions(-)


To https://github.com/elkomysara/simple_shell.git
   6611153..56fc096  main -> main


###
---
<center>
<h4> 2. File name</h4>
</center>

---


#### Write a program that prints the name of the file it was compiled from, followed by a new line.

* You are allowed to use the standard library

```bash
julien@ubuntu:~/0x0c. macro, structures$ gcc -Wall -pedantic -Werror -Wextra -std=gnu89 2-main.c -o c
julien@ubuntu:~/0x0c. macro, structures$ ./c 
2-main.c
julien@ubuntu:~/0x0c. macro, structures$ cp 2-main.c 02-main.c
julien@ubuntu:~/0x0c. macro, structures$ gcc -Wall -pedantic -Werror -Wextra -std=gnu89 02-main.c -o cc
julien@ubuntu:~/0x0c. macro, structures$ ./cc
02-main.c
julien@ubuntu:~/0x0c. macro, structures$   
```
#### Repo:
* GitHub repository: ``` alx-low_level_programming ```
* Directory: ```0x0D-preprocessor``` 
* File: ```2-main.c```

In [13]:
%%file alx/simple_shell2-main.c
#include <stdio.h>

/**
* main - prints the name of the file it was compiled from
*
* Return: Always 0.
*/
int main(void)
{
printf("%s\n", __FILE__);
return (0);
}


Overwriting 0x0D-preprocessor/2-main.c


In [15]:
%%bash
gcc -Wall -pedantic -Werror -Wextra -std=gnu89 alx/simple_shell2-main.c -o alx/simple_shellc
./alx/simple_shellc
cp alx/simple_shell2-main.c alx/simple_shell02-main.c
gcc -Wall -pedantic -Werror -Wextra -std=gnu89 alx/simple_shell02-main.c -o cc


0x0D-preprocessor/2-main.c


In [16]:
%%bash
git add .
git commit -m "alx-low_level_programming/alx/simple_shell0-putchar"
git push

[main d83673d] alx-low_level_programming/0x0D-preprocessor/0-putchar
 4 files changed, 24 insertions(+)
 create mode 100644 0x0D-preprocessor/02-main.c
 create mode 100644 0x0D-preprocessor/2-main.c
 create mode 100755 0x0D-preprocessor/c
 create mode 100755 cc


To https://github.com/elkomysara/alx-low_level_programming.git
   dbdf94f..d83673d  main -> main


###
---
<center>
<h4>3. Function-like macro</h4>
</center>

---


#### Write a function-like macro `ABS(x)` that computes the absolute value of a number `x`.

```bash
julien@ubuntu:~/0x0c. macro, structures$ cat 3-main.c
#include <stdio.h>
#include "3-function_like_macro.h"
#include "3-function_like_macro.h"

/**
 * main - check the code
 *
 * Return: Always 0.
 */
int main(void)
{
    int i;
    int j;

    i = ABS(-98) * 10;
    j = ABS(98) * 10;
    printf("%d, %d\n", i, j);
    return (0);
}
julien@ubuntu:~/0x0c. macro, structures$ gcc -Wall -pedantic -Werror -Wextra -std=gnu89 3-main.c -o d
julien@ubuntu:~/0x0c. macro, structures$ ./d 
980, 980
julien@ubuntu:~/0x0c. macro, structures$ 

```
#### Repo:
* GitHub repository: ``` alx-low_level_programming ```
* Directory: ```0x0D-preprocessor``` 
* File: ```3-function_like_macro.h```

In [18]:
%%file alx/simple_shell3-main.c
#include <stdio.h>
#include "3-function_like_macro.h"
#include "3-function_like_macro.h"

/**
 * main - check the code
 *
 * Return: Always 0.
 */
int main(void)
{
    int i;
    int j;

    i = ABS(-98) * 10;
    j = ABS(98) * 10;
    printf("%d, %d\n", i, j);
    return (0);
}

Overwriting 0x0D-preprocessor/3-main.c


In [19]:
%%file alx/simple_shell3-function_like_macro.h
#ifndef FUNCTION_LIKE_MACRO_H
#define FUNCTION_LIKE_MACRO_H

#define ABS(x) ((x) < 0 ? -(x) : (x))

#endif /* FUNCTION_LIKE_MACRO_H */


Writing 0x0D-preprocessor/3-function_like_macro.h


In [21]:
%%bash
gcc -Wall -pedantic -Werror -Wextra -std=gnu89 alx/simple_shell3-main.c -o alx/simple_shelld
./alx/simple_shelld


980, 980


In [22]:
%%bash
git add .
git commit -m "alx-low_level_programming/alx/simple_shell0-putchar"
git push

[main 4e412ae] alx-low_level_programming/0x0D-preprocessor/0-putchar
 3 files changed, 25 insertions(+)
 create mode 100644 0x0D-preprocessor/3-function_like_macro.h
 create mode 100644 0x0D-preprocessor/3-main.c
 create mode 100755 0x0D-preprocessor/d


To https://github.com/elkomysara/alx-low_level_programming.git
   d83673d..4e412ae  main -> main


###
---
<center>
<h4>4. SUM</h4>
</center>

---


#### Write a function-like macro `SUM(x, y)` that computes the sum of the numbers `x` and `y`.

```bash
 julien@ubuntu:~/0x0c. macro, structures$ cat 4-main.c
#include <stdio.h>
#include "4-sum.h"
#include "4-sum.h"

/**
 * main - check the code
 *
 * Return: Always 0.
 */
int main(void)
{
    int s;

    s = SUM(98, 1024);
    printf("%d\n", s);
    return (0);
}
julien@ubuntu:~/0x0c. macro, structures$ gcc -Wall -pedantic -Werror -Wextra -std=gnu89 4-main.c -o e
julien@ubuntu:~/0x0c. macro, structures$ ./e 
1122
julien@ubuntu:~/0x0c. macro, structures$ 
```
#### Repo:
* GitHub repository: ``` alx-low_level_programming ```
* Directory: ```0x0D-preprocessor``` 
* File: ```4-sim.h```

In [23]:
%%file alx/simple_shell4-sum.h

#ifndef SUM_H
#define SUM_H

#define SUM(x, y) ((x) + (y))

#endif /* SUM_H */


Writing 0x0D-preprocessor/4-sum.h


In [24]:
%%file alx/simple_shell4-main.c

#include <stdio.h>
#include "4-sum.h"
#include "4-sum.h"

/**
 * main - check the code
 *
 * Return: Always 0.
 */
int main(void)
{
    int s;

    s = SUM(98, 1024);
    printf("%d\n", s);
    return (0);
}



Writing 0x0D-preprocessor/4-main.c


In [25]:
%%bash
gcc -Wall -pedantic -Werror -Wextra -std=gnu89 alx/simple_shell4-main.c -o alx/simple_shelle
./alx/simple_shelle


1122


In [26]:
%%bash
git add .
git commit -m "alx-low_level_programming/alx/simple_shell0-putchar"
git push

[main ca563fe] alx-low_level_programming/0x0D-preprocessor/0-putchar
 3 files changed, 26 insertions(+)
 create mode 100644 0x0D-preprocessor/4-main.c
 create mode 100644 0x0D-preprocessor/4-sum.h
 create mode 100755 0x0D-preprocessor/e


To https://github.com/elkomysara/alx-low_level_programming.git
   4e412ae..ca563fe  main -> main
